In [5]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward(self, input):
        pass
    def backward(self, output_gradient, learning_rate):
        pass

In [10]:
import numpy as np
from scipy import signal

class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth): 
        # Lets say input is a RGB image of dimension 24 pixels (height) * 24 pixels (width)
        # So input_shape is 3 * 24 * 24
        # For simplicity, let's consider kernel_size is 2 * 2
        # Depth is the number of kernels we're using in each layer of the kernel
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        # Now output shape can be calculated using the formula Y = I - K + 1 on height and width of the input shape
        # Here Y is the shape of output matrix, I is the shape of input matrix and K is the shape of kernel
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size +1)
        # Lets say there are 2 layers of kernel and we know the depth of input image (input_depth) is 3, then depth is 2
        # Then the overall kernel shape is dept(2) * input_depth (3) * kernel_size(2*2) * kernel_size(2*2)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth): # In this case 2 (number of kernel layers)
            for j in range(self.input_depth): # In this case 3 (number of color channels)
                self.output[i] = signal.correlate2d(self.input[j], self.kernels[i,j], "valid")
        return self.output

